<a href="https://colab.research.google.com/github/nosale-arc/csr-gpt-rag/blob/main/CSR_RAG_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install --upgrade --user langchain_community langchain-openai pypdf chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython
import time

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import os
from langchain_community.document_loaders import PyPDFLoader

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
DOC_PATH = "/content/Nelson_Biology_unit_30d.pdf"
CHROMA_PATH = "nelson_db"

# load your pdf doc
loader = PyPDFLoader(DOC_PATH)
pages = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the doc into smaller chunks i.e. chunk_size=500
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(pages)

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('openai_key')

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# embed the chunks as vectors and load them into the database
db_chroma = Chroma.from_documents(chunks, embeddings, persist_directory=CHROMA_PATH)

def answer_question(query):

  # retrieve context - top 10 most relevant (closest) chunks to the query vector
  # (by default Langchain is using cosine distance metric)
  docs_chroma = db_chroma.similarity_search_with_score(query, k=10)

  # generate an answer based on given user query and retrieved context information
  context_text = "\n\n".join([doc.page_content for doc, _score in docs_chroma])

  # you can use a prompt template
  PROMPT_TEMPLATE = """
  Answer the question based only on the following context:
  {context}
  Answer the question based on the above context: {question}.
  Provide a detailed answer.
  Don’t justify your answers.
  Don’t give information not mentioned in the CONTEXT INFORMATION.
  Do not say "according to the context" or "mentioned in the context" or similar.
  Answer this question as if you were Steve Jobs
  """

  # load retrieved context and user query in the prompt template
  prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
  prompt = prompt_template.format(context=context_text, question=query)

  # call LLM model to generate the answer based on the given context and query
  model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
  response_text = model.predict(prompt)

  return response_text

In [ ]:
print(answer_question("What is the definition of a trait?"))

A trait is a specific characteristic or feature that is easily distinguishable in individuals, and each trait is controlled by a single recessive and dominant allele combination. In the context provided, traits such as blue eyes, widow's peak, hitchhiker's thumb, attached earlobes, freckles, dimples, tongue rolling, and long second toe are examples of traits that can be observed in human populations.
